In [1]:
import papermill as pm
import pandas as pd
import datetime
from shared.read_regulation import RegulationRetrievalHelper


In [2]:
import warnings
warnings.filterwarnings("ignore", 
                        category=DeprecationWarning, 
                        message=".*DataFrameGroupBy.apply operated on the grouping columns.*")

In [3]:
from config.config_loader import Config

config = Config()

In [4]:
alias = 'PR2'

In [5]:
festivos_path = config.get_path("data_path.elec.regulation.festivos")
periodos_cal_path = config.get_path("data_path.elec.regulation.calendario_periodos")

futures_path = config.get_path("data_path.elec.processed.futures")
spot_path = config.get_path("data_path.elec.processed.spot")
spot_formated_path = config.get_path("data_path.elec.processed.spot_formatted")

In [6]:
path_coefs_exc = config.get_path("data_path.elec.regulation.coefs_excesos")
path_ie = config.get_path("data_path.elec.regulation.ie")
path_ver_template = config.get_resolved_path("data_path.elec.verification_project.templates.verif")
path_estim_template = config.get_resolved_path("data_path.elec.verification_project.templates.estim")
compodem_a2_path = config.get_resolved_path("data_path.elec.verification_project.compodem_a2")

path_peajes_cargos_energia = config.get_resolved_path("data_path.elec.regulation.peajes_cargos_energia")
path_peajes_energia = config.get_resolved_path("data_path.elec.regulation.peajes_energia")
path_peajes_potencia = config.get_resolved_path("data_path.elec.regulation.peajes_potencia")
path_cargos_energia = config.get_resolved_path("data_path.elec.regulation.cargos_energia")
path_cargos_potencia = config.get_resolved_path("data_path.elec.regulation.cargos_potencia")
path_estructura_cargos = config.get_resolved_path("data_path.elec.regulation.estructura_cargos")
path_dto_peajes_electrointensivos = config.get_resolved_path("data_path.elec.regulation.dto_peajes_electrointensivos")

path_dto_peajes_electrointensivos


'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\regulation\\elec\\dto_peajes_electrointensivos.xlsx'

In [7]:
path_alias_elec = config.get_resolved_path("data_path.elec.customers.alias_elec")

In [8]:
# Notebooks
nb_elec_prices = config.get_resolved_path("notebooks.shared.elec_markets_2a")
nb_elec_perd = config.get_resolved_path("notebooks.shared.perdidas")
nb_elec_ssaa = config.get_resolved_path("notebooks.shared.ssaa")
nb_load = config.get_resolved_path("notebooks.load_compilation.export")

nb_market_matrix = config.get_resolved_path("notebooks.verification_project.build_matrix")
nb_market_matrix_estim = config.get_resolved_path("notebooks.verification_project.build_matrix_estim")
nb_contract_market_matrix = config.get_resolved_path("notebooks.verification_project.merge_market_contract")
nb_matrix_estim = config.get_resolved_path("notebooks.verification_project.build_matrix_estim")
nb_read_contracts = config.get_resolved_path("notebooks.verification_project.read_contract")
nb_estim = config.get_resolved_path("notebooks.verification_project.estimacion")
nb_estim_excel = config.get_resolved_path("notebooks.verification_project.estimacion_excel")
nb_invoices = config.get_resolved_path("notebooks.verification_project.hunt_invoices")
nb_verif = config.get_resolved_path("notebooks.verification_project.ver_nb")
nb_contract_market_matrix

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\notebooks\\verification_project\\4B_Merge_Contract_OMIE.ipynb'

In [9]:
db_elec_path = config.get_resolved_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")

path_load_parquet = config.get_resolved_path("data_path.elec.processed.load_parquet")
path_load_elec_folder = config.get_resolved_path("data_path.elec.load_compilation.folder")
db_elec_path

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\processed\\elec\\facturas_elec.db'

In [10]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

2025-07-04 14:07:14,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-04 14:07:14,911 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-04 14:07:14,911 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-04 14:07:14,911 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-04 14:07:14,911 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-04 14:07:14,923 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-04 14:07:14,924 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-04 14:07:14,947 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
df_alias_elec = pd.read_excel(path_alias_elec)

In [12]:
cliente = 'Taghleef'
year = 2025

df_alias_fil = df_alias_elec[df_alias_elec['cliente'] == cliente]
# df_alias_fil = df_alias_fil[df_alias_fil['alias'].isin(['WARNER BS', 'WARNER BQ', 'WARNER DW'])]
# df_alias_fil = df_alias_fil[~df_alias_fil['cups'].isin(['ES0022000009138571BZ'])]
df_alias_fil

,cliente,alias,cups,tarifa,electrointensivo,contrato
28,Taghleef,TAGHLEEF,ES0031101457946001KR,6.1 TD,True,1


In [13]:
df_load = pd.read_parquet(path_load_parquet)

In [14]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

class OptimizePotencia:
    
    def __init__(self,
                 df_load: pd.DataFrame,
                 df_kp: pd.DataFrame,
                 df_tep: pd.DataFrame,
                 df_cargos_potencia: pd.DataFrame,
                 df_peajes_potencia: pd.DataFrame,
                 potencia: list):
        
        self.df_kp = df_kp
        self.df_tep = df_tep
        self.df_cargos_potencia = df_cargos_potencia
        self.df_peajes_potencia = df_peajes_potencia
        self.potencia = potencia

        df = df_load.copy()
        # df['load'] = df['load'] * 4
        self.df_load = df

        self.days = len(df_load['datetime'].dt.date.unique())
        self._merge_load_coefs()

    def _merge_load_coefs(self):
        # Use self.df_load and self.df_kp
        df_merged = pd.merge(self.df_load, self.df_kp, on='periodo', how='left')
        df_merged = pd.merge(df_merged, self.df_tep, on='periodo', how='left')
        self.df_op = df_merged
        return self

    def excess_calc(self, group):
        # Calculate the squared excess only if load exceeds potencia
        group['input'] = group.apply(
            lambda row: (row['load'] - row['potencia']) ** 2 if (row['load'] - row['potencia'] > 0) else 0, 
            axis=1
        )
        # Sum the squared differences per period and calculate the weighted excess
        list_pw = group.groupby('periodo')['input'].sum()
        tep = group.groupby('periodo')['tep_power_exc'].mean()
        kp = group.groupby('periodo')['kp_power_exc'].mean()
        excesos_potencia = (np.sqrt(list_pw) * kp * tep).sum()
        return excesos_potencia

    def objective_function(self, potencia):
        # Create a DataFrame for power, assuming period indices from 1 to 6
        df_power = pd.DataFrame(potencia, index=[period for period in range(1, 7)])
        df_power.reset_index(inplace=True)
        df_power.columns = ['periodo', 'potencia']

        df = pd.merge(self.df_op, df_power, on='periodo', how='left')

        # Use self.df_cargos_potencia and self.df_peajes_potencia
        coste_potencia = ((self.df_cargos_potencia['cargos_potencia'] + self.df_peajes_potencia['peajes_potencia'])
                          * self.days * df_power['potencia']).sum()
        
        df["month"] = df["datetime"].dt.to_period("M")
        monthly_results = df.groupby("month").apply(self.excess_calc).reset_index(name="monthly_result")
        # print(monthly_results['monthly_result'])
        
        # Sum only the 'monthly_result' column to get a scalar value
        total_cost_excess = monthly_results['monthly_result'].sum()

        self.coste_termino_fijo = coste_potencia
        self.coste_excesos = total_cost_excess
        
        return total_cost_excess + coste_potencia
    
    def minimize_potencia(self):

        n = len(self.potencia)

        # Bounds: power values must be non-negative
        bounds = [(0, None)] * n

        # Constraint: each value must be less than or equal to the next (x[i+1] - x[i] ≥ 0)
        constraints = [
            {'type': 'ineq', 'fun': lambda x, i=i: x[i+1] - x[i]} for i in range(n - 1)
        ]

        result = minimize(
            self.objective_function,
            self.potencia,
            method="SLSQP",
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000, 'disp': True}
        )

        print(result)
        return result


In [16]:
df_optimizator = pd.DataFrame(columns=\
    ['cliente', 'alias'] + [f'potencia_p{i} op' for i in range(1,7)] + [f'potencia_p{i} old' for i in range(1,7)] + ['new_power_cost', 'new_cost_excess', 'old_power_cost', 'old_cost_excess', 'dif'])

print(len(df_optimizator.columns))

for index, row in df_alias_fil.iterrows():
    try:
        optimized_list = []
        # optimized_list.append(row['cliente'])
        # optimized_list.append(row['alias'])
        alias = row['alias']
        tarifa = row['tarifa']
        cups = row['cups']

        print(f"Optimizing power for alias: {alias}")
        Regulation = RegulationRetrievalHelper(
            tarifa=tarifa,
            path_peajes_cargos = path_peajes_cargos_energia,
            path_peajes_energia = path_peajes_energia,
            path_cargos_energia = path_cargos_energia,
            path_peajes_potencia = path_peajes_potencia,
            path_cargos_potencia = path_cargos_potencia,
            path_estructura_cargos = path_estructura_cargos,
            path_ie = path_ie,
            path_dto_peajes_electrointensivos = path_dto_peajes_electrointensivos,
            path_coefs_exc = path_coefs_exc
        )

        df_kp = Regulation._read_kp_power_exc_param(year=year)
        df_tep = Regulation._read_tep_power_exc_param(year=year)

        df_kp_2 = Regulation._read_kp_power_exc_param(year='2025 (2)')
        df_tep_2 = Regulation._read_tep_power_exc_param(year='2025 (2)')

        df_cargos_potencia = Regulation._read_peajes_slash_cargos(parameter='cargos_potencia',
                                            year=year)
        df_peajes_potencia = Regulation._read_peajes_slash_cargos(parameter='peajes_potencia',
                                            year=year)

        df_filtered_issued = facturas_df[
                                (facturas_df['cups'] == cups)
                                ]
        df_filtered_issued.sort_values(by='fin_periodo', ascending=True)
        df_filtered_issued = df_filtered_issued.tail(1)
        potencia = [df_filtered_issued[f'potencia_p{i+1}'].values[0] for i in range (6)]

        df_load_f = df_load[(df_load['resolution'] == 'quarter-hourly') &
                        (df_load['cups'] == cups)]
        
        # Unicamente se usan las dos líneas de debajo para Warner

        # df_load_f = df_load[(df_load['resolution'] == 'hourly') &
        #                 (df_load['cups'] == cups)]

        if df_load_f.empty:

            df_load_f = df_load[(df_load['resolution'] == 'hourly') &
                        (df_load['cups'] == cups)]
        else:
            df_load_f['load'] = df_load_f['load'] * 4
        df_load_f = df_load_f.copy()
        df_load_f.sort_values('datetime', ascending=True, inplace=True)

        date_fil = df_load_f['datetime'].max()
        print(date_fil)

        month_start = date_fil.month
        year_start = date_fil.year - 1
        date_start = datetime.datetime(date_fil.year - 1, month_start, 1,0,0)
        print(date_start)

        date_end = datetime.datetime(date_fil.year, month_start, 1,0,0) - datetime.timedelta(days=1)
        date_end = datetime.datetime.combine(date_end.date(),datetime.time(23,55))
        print(date_end)

        df_load_fil = df_load_f[
                        (df_load_f['datetime'] >= date_start) &
                        (df_load_f['datetime'] <= date_end)
                        ]
        len(df_load_fil['datetime'].dt.date.unique())

        OpNewReg = OptimizePotencia(
            df_load=df_load_fil,
            df_kp=df_kp_2,
            df_tep=df_tep_2,
            df_cargos_potencia=df_cargos_potencia,
            df_peajes_potencia=df_peajes_potencia,
            potencia=potencia
        )

        optimized_power_new = OpNewReg.minimize_potencia()
        POWEROPNEW = list(optimized_power_new.x)
        old_cost = OpNewReg.objective_function(potencia)
        old_power_cost = OpNewReg.coste_termino_fijo
        old_cost_excess = OpNewReg.coste_excesos
        optimized_cost = OpNewReg.objective_function(POWEROPNEW)
        new_power_cost = OpNewReg.coste_termino_fijo
        new_cost_excess = OpNewReg.coste_excesos
        dif = optimized_cost - old_cost

        items = [cliente, alias] + [P for P in POWEROPNEW] + [P for P in potencia]+ [new_power_cost, new_cost_excess, old_power_cost, old_cost_excess, dif]
        for item in items:
            optimized_list.append(item)
        print(len(items))
        df_optimizator = pd.concat([df_optimizator, pd.DataFrame([optimized_list], columns=df_optimizator.columns)], ignore_index=True)
    except:
        print(f"Error processing alias: {alias}")
        continue

df_optimizator.to_excel(f'Optimizacion potencia {cliente} {year}.xlsx', index=False)
print(df_optimizator)

19
Optimizing power for alias: TAGHLEEF
2025-06-01 23:00:00
2024-06-01 00:00:00
2025-05-31 23:55:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 226922.96285205564
            Iterations: 33
            Function evaluations: 231
            Gradient evaluations: 33
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 226922.96285205564
       x: [ 3.740e+03  3.756e+03  3.756e+03  3.756e+03  3.756e+03
            3.756e+03]
     nit: 33
     jac: [ 0.000e+00 -4.494e+00  5.254e-01  1.459e+00  1.912e+00
            5.977e-01]
    nfev: 231
    njev: 33
19
    cliente     alias  potencia_p1 op  potencia_p2 op  potencia_p3 op  \
0  Taghleef  TAGHLEEF     3739.849856     3755.618042     3755.618042   

   potencia_p4 op  potencia_p5 op  potencia_p6 op  potencia_p1 old  \
0     3755.618042     3755.618042     3755.618042           4030.0   

   potencia_p2 old  potencia_p3 old  potencia_p4 old  potencia_p5 old  \
0   

C:\Users\mikel.perez\AppData\Local\Temp\ipykernel_5976\705448792.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_optimizator = pd.concat([df_optimizator, pd.DataFrame([optimized_list], columns=df_optimizator.columns)], ignore_index=True)
